# Model Evaluation

In [1]:
import plaidml.keras as pk
pk.install_backend()

import pandas as pd
import numpy as np
import joblib
import pickle
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [2]:
with open('./sequence_data.pickle', 'rb') as f:
    sequence_data = pickle.load(f)
    
with open('./numerical_data.pickle', 'rb') as f:
    numeric_data = pickle.load(f)
    
X_seq, y = sequence_data
X_num, y = numeric_data

X = np.concatenate([X_num, X_seq], axis=1)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.33)

num_train = x_train[:, :8]
seq_train = x_train  

num_test = x_test[:, :8]
seq_test = x_test  

xc_train = [num_train, seq_train] 
xc_test = [num_test, seq_test] 

In [3]:
def false_neg(preds, y_test):
    fn = 0
    for i in range(len(preds)): 
        if preds[i]==0 and y_test[i]!=preds[i]:
            fn += 1

    return fn / len(y_test)

def evaluate(preds, y_test):
    print(f'Accuracy Score: {round(accuracy_score(preds, y_test)*100, 2)}%')
    print(f'AUC ROC Score: {round(roc_auc_score(preds, y_test)*100, 2)}%')
    print('\n__CLASSIFICATION REPORT__:')
    print(classification_report(preds, y_test))
    print('\n__CONFUSION MATRIX__:')
    print(confusion_matrix(preds, y_test)/len(y_test))
    print('\n__FALSE NEGATIVE RATE__:')
    print(f'{round(false_neg(preds, y_test)*100,2)}%')

In [4]:
# Random Forest
rf = joblib.load('./random_forest_model.2.joblib')

# combined deep learning model
cm = load_model('combined_cnn_mlp_model.3.1.917.h5')

INFO:plaidml:Opening device "metal_amd_radeon_pro_570x.0"


### Random Forest Evaluation

In [5]:
rf_preds = rf.predict(x_test)
evaluate(rf_preds, y_test)

Accuracy Score: 94.58%
AUC ROC Score: 94.63%

__CLASSIFICATION REPORT__:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95     11706
           1       0.93      0.96      0.94     10895

    accuracy                           0.95     22601
   macro avg       0.95      0.95      0.95     22601
weighted avg       0.95      0.95      0.95     22601


__CONFUSION MATRIX__:
[[0.48250077 0.03544091]
 [0.01876023 0.46329808]]

__FALSE NEGATIVE RATE__:
3.54%


### Deep Learning Evaluation

In [6]:
input_data = [x_test[:, :8], x_test[:, 8:]]
cm_preds = cm.predict(input_data)
evaluate(np.rint(cm_preds), y_test)

Accuracy Score: 92.55%
AUC ROC Score: 92.57%

__CLASSIFICATION REPORT__:
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93     11550
         1.0       0.92      0.93      0.92     11051

    accuracy                           0.93     22601
   macro avg       0.93      0.93      0.93     22601
weighted avg       0.93      0.93      0.93     22601


__CONFUSION MATRIX__:
[[0.4689173  0.04212203]
 [0.0323437  0.45661696]]

__FALSE NEGATIVE RATE__:
4.21%
